In [ ]:
from IPython.display import display, HTML
from vulcan_rwd_ig.core import *

# Overview

The goal of this notebook is to build a notional cohort using data available in the ips.health server - so that we can look at IPS data from the perspective of ansering research questions.

# Inclusion/Exclusion

The patients for this study would have the following criteria (Inclusion):
- been vaccinated with mRNA SARS-COV-2 vaccine
- been given prednisone after vaccination
- observed monocytes in blood 0.2 to 0.4 × 10\*9 per liter after taking prednisone

The patients for this study would not have the following criteria (Exclusion):
- any [allergies](https://build.fhir.org/allergyintolerance.html)
    - Rebecca reminded us of the Sagan quote; Absence of Evidence does not mean Evidence of Absence
    - TODO: how can we access `List.emptyReason`

In [ ]:
client = FhirClient('http://ips.health:8080/fhir')

In [ ]:
# http://ips.health:8080/fhir/Immunization?vaccine-code=208&_pretty=true
immunizations = client.get_all_resources('Immunization', {
    'status': 'completed',
    'vaccine-code': '208'
})

GET data\cache\ips.health-8080\fhirImmunization-30 from local cache
Returning 4 entries


In [ ]:
# http://ips.health:8080/fhir/Observation?code=http://loinc.org|26484-6&_pretty=true
observations = client.get_all_resources('Observation', {
    'code': 'http://loinc.org|26484-6'
})

GET data\cache\ips.health-8080\fhirObservation-33 from local cache
Returning 4 entries


In [ ]:
patient_ids = intersection_patient_ids(immunizations, observations)
patient_ids

['Patient/f15d2419-fbff-464a-826d-0afe8f095771']

In [ ]:
patient_id = patient_ids[0]
patient_summary = client.get_as_raw_json(f'{patient_id}/$summary')

GET data\cache\ips.health-8080\fhirPatientf15d2419-fbff-464a-826d-0afe8f095771$summary-6 from local cache


The following data points (shown in the summary below) show all inclusion/exclusion criteria are met

- no allergies
- vaccinated 2021-08-05 and 2022-08-27
- prednisone 2021-12-17
- Monocytes 0.3 2021-12-21

In [ ]:
display(HTML(f'<h1>Patient ID: {patient_id}</h1>'))
for div in find_by_key(patient_summary, 'div'):
    display(HTML(div))

Medical Problems,Status,Comments,Onset Date


Status,FINAL


Status,FINAL


Status,FINAL


Status,FINAL


...

Status,FINAL


Status,FINAL


Status,FINAL


Status,FINAL


Status,FINAL


# medications can be hard to search for ...

In [ ]:
# http://ips.health:8080/fhir/MedicationStatement?medication.code=10060441000116103&_pretty=true
client.get_as_raw_json('MedicationStatement', {
#     'status': 'completed', # the medication we want is active
#     '_include': 'Medication:medicationReference', # _include and _revinclude can only be specified for defined resource search parameters
    'medication.code': 'http://nzmt.org.nz|10060441000116103'})

GET data\cache\ips.health-8080\fhirMedicationStatement-44 from local cache


{'resourceType': 'Bundle',
 'id': '8aa019e2-a75a-4966-a1eb-fcda6809cb41',
 'meta': {'lastUpdated': '2022-11-08T15:33:09.047+00:00'},
 'type': 'searchset',
 'total': 1,
 'link': [{'relation': 'self',
   'url': 'http://ips.health:8080/fhir/MedicationStatement?medication.code=http%3A%2F%2Fnzmt.org.nz%7C10060441000116103'}],
 'entry': [{'fullUrl': 'http://ips.health:8080/fhir/MedicationStatement/1368a5ba-644d-44cd-8a5b-eae138d7047a',
   'resource': {'resourceType': 'MedicationStatement',
    'id': '1368a5ba-644d-44cd-8a5b-eae138d7047a',
    'meta': {'versionId': '1',
     'lastUpdated': '2022-09-27T15:22:47.372+00:00',
     'source': '#0GPrPkmJ1EEG3ziH'},
    'status': 'active',
    'medicationReference': {'reference': 'Medication/8d85a90a-833e-4869-b4a7-b7b26c35880c'},
    'subject': {'reference': 'Patient/f15d2419-fbff-464a-826d-0afe8f095771'},
    'effectiveDateTime': '2021-12-17',
    'informationSource': {'reference': 'Organization/90862041-4bfd-45d4-b51a-beaedcd5a012'},
    'dosage':

In [ ]:
medication_statements = client.get_all_resources('MedicationStatement', dict(patient=patient_id))
for medication_statement in medication_statements:
    medication_statement['_medicationReference'] = medication_statement['medicationReference']
    medication_statement['medicationReference'] = client.get_as_raw_json(medication_statement['medicationReference.reference'])
medication_statements[0]

GET data\cache\ips.health-8080\fhirMedicationStatement-22 from local cache
Returning 2 entries
GET data\cache\ips.health-8080\fhirMedication8d85a90a-833e-4869-b4a7-b7b26c35880c-36 from local cache
GET data\cache\ips.health-8080\fhirMedication9ad9c51a-af38-46b5-b22c-23aa92818e58-35 from local cache


{'resourceType': 'MedicationStatement', 'id': '1368a5ba-644d-44cd-8a5b-eae138d7047a', 'meta': {'versionId': '1', 'lastUpdated': '2022-09-27T15:22:47.372+00:00', 'source': '#0GPrPkmJ1EEG3ziH'}, 'status': 'active', 'medicationReference': {'resourceType': 'Medication', 'id': '8d85a90a-833e-4869-b4a7-b7b26c35880c', 'meta': {'versionId': '1', 'lastUpdated': '2022-09-27T15:22:14.437+00:00', 'source': '#7wxneUdRW82ITIFy'}, 'text': {'status': 'generated', 'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText"> prednisone 1 mg tablet </div></div>'}, 'code': {'coding': [{'system': 'http://nzmt.org.nz', 'code': '10060441000116103', 'display': 'prednisone 1 mg tablet'}, {'system': 'http://snomed.info/sct', 'code': '10312003', 'display': 'Product containing prednisone (medicinal product)'}]}, 'form': {'coding': [{'system': 'http://snomed.info/sct', 'code': '385055001', 'display': 'tablet'}]}}, 'subject': {'reference': 'Patient/f15d2419-fbff-464a-826d-0afe8f095771'}, 'effectiv

# patient or subject ...

In [ ]:
immunizations[0]['patient']

{'reference': 'Patient/f15d2419-fbff-464a-826d-0afe8f095771'}

In [ ]:
observations[0]['subject']

{'reference': 'Patient/f15d2419-fbff-464a-826d-0afe8f095771'}

# TODO: compare `$summary` with `$everything`

# What if we just want to look at lots of patient summaries?

In [ ]:
client = FhirClient('http://ips.health:8080/fhir')
patient_resources = client.get_all_resources('Patient', {})

GET data\cache\ips.health-8080\fhirPatient-0 from local cache
Returning 13 entries


In [ ]:
patient_summaries = {}
for patient_resource in patient_resources:
    patient_id = patient_resource['id']
    patient_summaries[patient_id] = client.get_as_raw_json(f'Patient/{patient_id}/$summary')

GET data\cache\ips.health-8080\fhirPatient98549f1a-e0d5-4454-849c-f5b97d3ed299$summary-1 from local cache
GET data\cache\ips.health-8080\fhirPatient6c968ec2-5e95-4d4b-87f9-0c2b9ac99a24$summary-2 from local cache
GET data\cache\ips.health-8080\fhirPatient274261ba-23d0-4822-a5f9-9d4b9296eea3$summary-3 from local cache
GET data\cache\ips.health-8080\fhirPatiente1efb218-12c3-4b51-80fb-3f4cdc81e11d$summary-4 from local cache
GET data\cache\ips.health-8080\fhirPatient6a670b6a-e334-4ec6-a684-0c5ce66a1554$summary-5 from local cache
GET data\cache\ips.health-8080\fhirPatientf15d2419-fbff-464a-826d-0afe8f095771$summary-6 from local cache
GET data\cache\ips.health-8080\fhirPatient1b509aba-6d71-47b5-bb5d-bd38786a3163$summary-7 from local cache
GET data\cache\ips.health-8080\fhirPatientd58dd75b-cf09-4a1c-b913-c9e867f27616$summary-8 from local cache
GET data\cache\ips.health-8080\fhirPatient46cbbd6a-0b2d-4be1-bdab-d9fda1a0d3e7$summary-9 from local cache
GET data\cache\ips.health-8080\fhirPatient11f4

In [ ]:
for patient_id, patient_summary in patient_summaries.items():
    display(HTML(f'<h1>Patient ID: {patient_id}</h1>'))
    for div in find_by_key(patient_summary, 'div'):
        display(HTML(div))

Medical Problems,Status,Comments,Onset Date


Identifier,maria-1
Date of birth,05 May 1946


Medical Problems,Status,Comments,Onset Date


-,Relationship,Name,Telecom,Address
*,mother (RoleCode#MTH),Martha Mum,+33-555-20036,Promenade des Anglais 111 Lyon 69001 FR


Medical Problems,Status,Comments,Onset Date


...

Status,FINAL
Issued,13 June 2016 00:20:27


Status,FINAL
Issued,13 June 2016 00:20:27


Status,FINAL
Issued,07 July 2014 00:20:27


Status,FINAL
Issued,02 July 2012 00:20:27


Medical Problems,Status,Comments,Onset Date
